<a href="https://colab.research.google.com/github/VictoriaRe/HSE-DataScience/blob/master/Assignment9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
!wget -O data.zip https://github.com/thedenaas/hse_seminars/blob/master/2018/seminar_13/data.zip?raw=true
!unzip data.zip

--2020-03-22 17:29:17--  https://github.com/thedenaas/hse_seminars/blob/master/2018/seminar_13/data.zip?raw=true
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/thedenaas/hse_seminars/raw/master/2018/seminar_13/data.zip [following]
--2020-03-22 17:29:18--  https://github.com/thedenaas/hse_seminars/raw/master/2018/seminar_13/data.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/thedenaas/hse_seminars/master/2018/seminar_13/data.zip [following]
--2020-03-22 17:29:18--  https://raw.githubusercontent.com/thedenaas/hse_seminars/master/2018/seminar_13/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.1

In [0]:
with open('data.txt', 'r', encoding='utf-8') as txt_file:
    data = txt_file.read()

In [77]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
def tokenize(data):
  return nltk.word_tokenize(data)

In [0]:
with open("stopwords.txt", "r" ) as f:
    stopwords=[line.strip().lower() for line in f.readlines()]

In [0]:
import pandas

In [0]:
train_data = pandas.DataFrame()
train_data['positive'] = nltk.sent_tokenize(data)[:800] 

In [0]:
train_data['negative_1'] = train_data['positive'].apply(lambda n: train_data.iloc[np.random.choice(800), 0])

In [95]:
train_data

,positive,negative_1
0,Barclays' defiance of US fines has merit Barcl...,"The Dallas police chief, David Brown, recently..."
1,"So it is tempting to think the bank, when aske...",It was one of the first email services not tie...
2,"That is not the view of the chief executive, J...","The suspension of Spencer, along with the acco..."
3,Barclays thinks the DoJ’s claims are “disconne...,"In December, Defense Secretary Ashton Carter a..."
4,"But actually, some grudging respect for Staley...","Low interest rates, accompanied by piles of go..."
...,...,...
795,Frances Perraudin had some more details earlier.,“I think Bernie ought to be president – of Ben...
796,"Remain campaigners in Islington, a Labour stro...",“Had the campaign known of her full filmograph...
797,"The ’s David Pegg, who is anchored in deepest ...",Barclays thinks the DoJ’s claims are “disconne...
798,“Turnout appears to have been fairly high.,The Italian referendum proved a non-event for ...


In [0]:
train_data['negative_2'] = train_data['positive'].apply(lambda n: train_data.iloc[np.random.choice(800), 0])
train_data['negative_3'] = train_data['positive'].apply(lambda n: train_data.iloc[np.random.choice(800), 0])

In [97]:
train_data

,positive,negative_1,negative_2,negative_3
0,Barclays' defiance of US fines has merit Barcl...,"The Dallas police chief, David Brown, recently...","He was born in Florida, but spent some of his ...",Lotus Notes launched - 1989 The first version ...
1,"So it is tempting to think the bank, when aske...",It was one of the first email services not tie...,The DoJ lawsuit says Barclays employees called...,The soundtrack to that ad is considered one of...
2,"That is not the view of the chief executive, J...","The suspension of Spencer, along with the acco...",Until that point pre-capacitive consumer smart...,When we temporarily suspend multiple accounts ...
3,Barclays thinks the DoJ’s claims are “disconne...,"In December, Defense Secretary Ashton Carter a...",They’re writing off March 8.,“All this new knowledge is useful in the sense...
4,"But actually, some grudging respect for Staley...","Low interest rates, accompanied by piles of go...",It is understood that most clubs largely agree...,"Most stock indices are up in early trading, le..."
...,...,...,...,...
795,Frances Perraudin had some more details earlier.,“I think Bernie ought to be president – of Ben...,"Yes, even after two years of exposure to the a...","It is immoral.” In the aftermath, Senator Mike..."
796,"Remain campaigners in Islington, a Labour stro...",“Had the campaign known of her full filmograph...,European Union referendum polling day – as it ...,On a weekend that saw yet another mass shootin...
797,"The ’s David Pegg, who is anchored in deepest ...",Barclays thinks the DoJ’s claims are “disconne...,Fitch said: The Negative Outlook for the Itali...,"“Serve your community, don’t be a part of the ..."
798,“Turnout appears to have been fairly high.,The Italian referendum proved a non-event for ...,It is understood that while no vote was taken ...,"In the league, City have not had consecutive v..."


In [0]:
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset, Iterator

TEXT = Field(include_lengths=False, 
             batch_first=True, 
             tokenize=tokenize,
             lower=True,
             stop_words=stopwords)

datafields = [('text',TEXT)]

In [0]:
train_data.to_csv('dataset.csv', index=False)
train = TabularDataset(path="dataset.csv", format='csv',
                     skip_header=True, fields=datafields)

In [0]:
TEXT.build_vocab(train)

In [0]:
train_iter = Iterator(train, 256, shuffle = True)

In [0]:
import torch
import torch.nn as nn

Имплементация взята отсюда: https://github.com/alexeyev/abae-pytorch

In [0]:
# -*- coding: utf-8 -*-
import numpy as np
import torch
from torch.nn import init
from torch.nn.parameter import Parameter


class SelfAttention(torch.nn.Module):
    def __init__(self, wv_dim, maxlen):
        super(SelfAttention, self).__init__()
        self.wv_dim = wv_dim

        # max sentence length -- batch 2nd dim size
        self.maxlen = maxlen
        self.M = Parameter(torch.Tensor(wv_dim, wv_dim))
        init.kaiming_uniform(self.M.data)

        # softmax for attending to wod vectors
        self.attention_softmax = torch.nn.Softmax()

    def forward(self, input_embeddings):
        # (b, wv, 1)
        mean_embedding = torch.mean(input_embeddings, (1,)).unsqueeze(2)

        # (wv, wv) x (b, wv, 1) -> (b, wv, 1)
        product_1 = torch.matmul(self.M, mean_embedding)

        # (b, maxlen, wv) x (b, wv, 1) -> (b, maxlen, 1)
        product_2 = torch.matmul(input_embeddings, product_1).squeeze(2)

        results = self.attention_softmax(product_2)

        return results

    def extra_repr(self):
        return 'wv_dim={}, maxlen={}'.format(self.wv_dim, self.maxlen)


class ABAE(torch.nn.Module):
    """
        The model described in the paper ``An Unsupervised Neural Attention Model for Aspect Extraction''
        by He, Ruidan and  Lee, Wee Sun  and  Ng, Hwee Tou  and  Dahlmeier, Daniel, ACL2017
        https://aclweb.org/anthology/papers/P/P17/P17-1036/

    """

    def __init__(self, wv_dim=200, asp_count=30, ortho_reg=0.1, maxlen=201, init_aspects_matrix=None):
        """
        Initializing the model

        :param wv_dim: word vector size
        :param asp_count: number of aspects
        :param ortho_reg: coefficient for tuning the ortho-regularizer's influence
        :param maxlen: sentence max length taken into account
        :param init_aspects_matrix: None or init. matrix for aspects
        """
        super(ABAE, self).__init__()
        self.wv_dim = wv_dim
        self.asp_count = asp_count
        self.ortho = ortho_reg
        self.maxlen = maxlen

        self.attention = SelfAttention(wv_dim, maxlen)
        self.linear_transform = torch.nn.Linear(self.wv_dim, self.asp_count)
        self.softmax_aspects = torch.nn.Softmax()
        self.aspects_embeddings = Parameter(torch.Tensor(wv_dim, asp_count))

        if init_aspects_matrix is None:
            torch.nn.init.xavier_uniform(self.aspects_embeddings)
        else:
            self.aspects_embeddings.data = torch.from_numpy(init_aspects_matrix.T)

    def get_aspects_importances(self, text_embeddings):
        """
            Takes embeddings of a sentence as input, returns attention weights
        """

        # compute attention scores, looking at text embeddings average
        attention_weights = self.attention(text_embeddings)

        # multiplying text embeddings by attention scores -- and summing
        # (matmul: we sum every word embedding's coordinate with attention weights)
        weighted_text_emb = torch.matmul(attention_weights.unsqueeze(1), # (batch, 1, sentence)
                                         text_embeddings                 # (batch, sentence, wv_dim)
                                         ).squeeze()

        # encoding with a simple feed-forward layer (wv_dim) -> (aspects_count)
        raw_importances = self.linear_transform(weighted_text_emb)

        # computing 'aspects distribution in a sentence'
        aspects_importances = self.softmax_aspects(raw_importances)

        return attention_weights, aspects_importances, weighted_text_emb

    def forward(self, text_embeddings, negative_samples_texts):

        # negative samples are averaged
        averaged_negative_samples = torch.mean(negative_samples_texts, dim=2)

        # encoding: words embeddings -> sentence embedding, aspects importances
        _, aspects_importances, weighted_text_emb = self.get_aspects_importances(text_embeddings)

        # decoding: aspects embeddings matrix, aspects_importances -> recovered sentence embedding
        recovered_emb = torch.matmul(self.aspects_embeddings, aspects_importances.unsqueeze(2)).squeeze()

        # loss
        reconstruction_triplet_loss = ABAE._reconstruction_loss(weighted_text_emb,
                                                                recovered_emb,
                                                                averaged_negative_samples)
        max_margin = torch.max(reconstruction_triplet_loss, torch.zeros_like(reconstruction_triplet_loss))

        return self.ortho * self._ortho_regularizer() + max_margin

    @staticmethod
    def _reconstruction_loss(text_emb, recovered_emb, averaged_negative_emb):

        positive_dot_products = torch.matmul(text_emb.unsqueeze(1), recovered_emb.unsqueeze(2)).squeeze()
        negative_dot_products = torch.matmul(averaged_negative_emb, recovered_emb.unsqueeze(2)).squeeze()
        reconstruction_triplet_loss = torch.sum(1 - positive_dot_products.unsqueeze(1) + negative_dot_products, dim=1)

        return reconstruction_triplet_loss

    def _ortho_regularizer(self):
        return torch.norm(
            torch.matmul(self.aspects_embeddings.t(), self.aspects_embeddings) \
            - torch.eye(self.asp_count))

    def get_aspect_words(self, w2v_model, topn=15):
        words = []

        # getting aspects embeddings
        aspects = self.aspects_embeddings.detach().numpy()

        # getting scalar products of word embeddings and aspect embeddings;
        # to obtain the ``probabilities'', one should also apply softmax
        words_scores = w2v_model.wv.syn0.dot(aspects)

        for row in range(aspects.shape[1]):
            argmax_scalar_products = np.argsort(- words_scores[:, row])[:topn]
            # print([w2v_model.wv.index2word[i] for i in argmax_scalar_products])
            # print([w for w, dist in w2v_model.similar_by_vector(aspects.T[row])[:topn]])
            words.append([w2v_model.wv.index2word[i] for i in argmax_scalar_products])

        return words
